# CONFIG: Building Quantum Circuits

Welcome to quantum circuit construction! In this notebook, you'll learn how to build complex quantum circuits with multiple qubits.

## DOCS: Learning Objectives

- Multi-qubit systems and entanglement
- Two-qubit gates (CNOT, CZ, SWAP)
- Circuit composition and design patterns
- Bell states and GHZ states
- Quantum teleportation protocol

Let's build some amazing quantum circuits! 

In [ ]:
# Essential imports
from quantumsim import Circuit, Executor
from quantumsim.viz.ascii import ASCIIDrawer
from quantumsim.viz.advanced import QuantumCircuitAnalyzer
import numpy as np
import matplotlib.pyplot as plt

print("CONFIG: Building Quantum Circuits")
print("=" * 40)

executor = Executor()

## 1. Multi-Qubit Systems

When we have multiple qubits, the quantum state space grows exponentially!

- 1 qubit: 2 states (|0, |1)
- 2 qubits: 4 states (|00, |01, |10, |11) 
- 3 qubits: 8 states (|000, |001, ..., |111)
- n qubits: 2ⁿ states

Let's explore this:

In [ ]:
# Create multi-qubit systems
for n_qubits in [1, 2, 3, 4]:
 circuit = Circuit(n_qubits)
 # All qubits start in |00...0 state
 
 state = executor.run(circuit)
 n_amplitudes = len(state.data)
 
 print(f"STATS: {n_qubits} qubit(s): {n_amplitudes} amplitudes")
 if n_qubits <= 2:
 print(f" State: {state.data}")
 else:
 print(f" First few amplitudes: {state.data[:4]}...")
 print()

## 2. Two-Qubit Gates: Creating Entanglement

The real power of quantum computing comes from entangling qubits!

In [ ]:
# CNOT Gate (Controlled-X)
print("TARGET: CNOT Gate (Controlled-X)")
cnot_circuit = Circuit(2)
cnot_circuit.h(0) # Put control qubit in superposition
cnot_circuit.cx(0, 1) # CNOT: control=0, target=1

# Visualize the circuit
drawer = ASCIIDrawer(cnot_circuit)
print("Circuit:")
print(drawer.draw())

# Execute and analyze
bell_state = executor.run(cnot_circuit)
print(f"\nBell state: {bell_state.data}")
print("This is maximally entangled! |00 + |11")

# Measure to see correlations
measurements = bell_state.measure_all(shots=1000)
print(f"\nMeasurements: {measurements}")
print("Notice: Only |00 and |11 appear - perfect correlation!")

In [ ]:
# Let's see what happens without the Hadamard
print("SEARCH: CNOT without superposition:")
cnot_only = Circuit(2)
cnot_only.cx(0, 1) # CNOT on |00

state_no_h = executor.run(cnot_only)
print(f"State: {state_no_h.data}")
print("Result: Still |00 (CNOT doesn't flip if control is |0)")

# With control qubit in |1
print("\nSEARCH: CNOT with control in |1:")
cnot_flip = Circuit(2)
cnot_flip.x(0) # Put control in |1
cnot_flip.cx(0, 1) # CNOT flips target

state_flip = executor.run(cnot_flip)
print(f"State: {state_flip.data}")
print("Result: |11 (CNOT flips target when control is |1)")

## 3. The Four Bell States

Bell states are the fundamental two-qubit entangled states. Let's create all four!

In [ ]:
# The four Bell states
bell_states = [
 ("Φ⁺", lambda c: c.h(0).cx(0, 1)), # |00 + |11
 ("Φ⁻", lambda c: c.h(0).cx(0, 1).z(0)), # |00 - |11 
 ("Ψ⁺", lambda c: c.h(0).cx(0, 1).x(1)), # |01 + |10
 ("Ψ⁻", lambda c: c.h(0).cx(0, 1).x(1).z(0)) # |01 - |10
]

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('BELL: The Four Bell States', fontsize=16)

for i, (name, state_prep) in enumerate(bell_states):
 circuit = Circuit(2)
 state_prep(circuit)
 
 state = executor.run(circuit)
 measurements = state.measure_all(shots=1000)
 
 # Plot measurements
 ax = axes[i//2, i%2]
 states = list(measurements.keys())
 counts = list(measurements.values())
 
 bars = ax.bar(states, counts, alpha=0.7, 
 color=['blue', 'red', 'green', 'orange'][:len(states)])
 ax.set_title(f'Bell State |{name}')
 ax.set_xlabel('Measurement Outcome')
 ax.set_ylabel('Count (out of 1000)')
 ax.set_ylim(0, 600)
 
 # Add value labels on bars
 for bar, count in zip(bars, counts):
 ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,
 str(count), ha='center', va='bottom')
 
 print(f"Bell state |{name}: {measurements}")

plt.tight_layout()
plt.show()

## 4. Multi-Qubit Entanglement: GHZ State

Let's create a three-qubit entangled state - the famous GHZ state!

In [ ]:
# GHZ State: |000 + |111
print("SPARKLE: Creating GHZ State (3-qubit entanglement)")

ghz_circuit = Circuit(3)
ghz_circuit.h(0) # Superposition on qubit 0
ghz_circuit.cx(0, 1) # Entangle qubits 0 and 1
ghz_circuit.cx(1, 2) # Entangle qubit 2

# Visualize
drawer = ASCIIDrawer(ghz_circuit)
print("GHZ Circuit:")
print(drawer.draw())

# Execute
ghz_state = executor.run(ghz_circuit)
print(f"\nGHZ State: {ghz_state.data}")

# Measure
measurements = ghz_state.measure_all(shots=1000)
print(f"\nMeasurements: {measurements}")
print("Notice: Only |000 and |111 - all qubits are correlated!")

In [ ]:
# Visualize GHZ state measurements
states = list(measurements.keys())
counts = list(measurements.values())

plt.figure(figsize=(10, 6))
bars = plt.bar(states, counts, alpha=0.7, color='purple')
plt.title('SPARKLE: GHZ State Measurements (3-qubit entanglement)')
plt.xlabel('Measurement Outcome')
plt.ylabel('Count (out of 1000)')

# Add value labels
for bar, count in zip(bars, counts):
 plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
 str(count), ha='center', va='bottom', fontweight='bold')

plt.show()

print("BULB: The GHZ state shows that all three qubits are perfectly correlated!")
print(" Measuring one qubit instantly determines the others.")

## 5. Circuit Analysis and Optimization

Let's analyze the complexity of our quantum circuits:

In [ ]:
# Analyze different circuits
circuits_to_analyze = [
 ("Single Qubit", Circuit(1).h(0)),
 ("Bell State", Circuit(2).h(0).cx(0, 1)),
 ("GHZ State", ghz_circuit),
]

print("STATS: Circuit Analysis")
print("=" * 50)

for name, circuit in circuits_to_analyze:
 analysis = QuantumCircuitAnalyzer.analyze_circuit(circuit)
 
 print(f"\nCONFIG: {name}:")
 print(f" Qubits: {analysis['num_qubits']}")
 print(f" Gates: {analysis['num_gates']}")
 print(f" Depth: {analysis['depth']}")
 print(f" Gate types: {analysis['gate_counts']}")
 print(f" Entangling gates: {analysis['entangling_gates']}")

## 6. Quantum Teleportation Protocol

Now let's build the famous quantum teleportation circuit!

In [ ]:
# Quantum Teleportation: Send an unknown quantum state from Alice to Bob
print(" Quantum Teleportation Protocol")
print("=" * 40)

# Step 1: Prepare the state to be teleported (Alice's qubit)
# Let's teleport the |+ state
teleport_circuit = Circuit(3)
teleport_circuit.h(0) # Alice's qubit in |+ state to be teleported

print("Step 1: Alice's qubit prepared in |+ state")

# Step 2: Create Bell pair between Alice and Bob (qubits 1 and 2)
teleport_circuit.h(1) # Create superposition
teleport_circuit.cx(1, 2) # Entangle qubits 1 (Alice) and 2 (Bob)

print("Step 2: Bell pair created between Alice and Bob")

# Step 3: Alice's Bell measurement (joint measurement of qubits 0 and 1)
teleport_circuit.cx(0, 1) # Entangle Alice's qubits
teleport_circuit.h(0) # Hadamard on the teleported qubit

print("Step 3: Alice performs Bell measurement")

# Visualize the complete circuit
drawer = ASCIIDrawer(teleport_circuit)
print("\n Teleportation Circuit:")
print(drawer.draw())
print("\nQubit 0: Alice's state to teleport")
print("Qubit 1: Alice's half of Bell pair") 
print("Qubit 2: Bob's half of Bell pair")

In [ ]:
# Execute teleportation and analyze results
teleport_state = executor.run(teleport_circuit)
measurements = teleport_state.measure_all(shots=1000)

print(" Teleportation Results:")
print(f"Measurements: {measurements}")

# Analyze the measurement patterns
print("\nSEARCH: Analysis:")
print("In real teleportation, Alice would:")
print("1. Measure qubits 0 and 1 (getting 00, 01, 10, or 11)")
print("2. Send the classical bits to Bob") 
print("3. Bob applies corrections based on Alice's result:")
print(" - 00: Do nothing")
print(" - 01: Apply X gate")
print(" - 10: Apply Z gate") 
print(" - 11: Apply X then Z")
print("\nDIZZY: After corrections, Bob has Alice's original |+ state!")

## 7. Circuit Design Patterns

Let's learn some useful quantum circuit patterns:

In [ ]:
# Pattern 1: Quantum Fourier Transform (simplified 2-qubit version)
print("WAVE: Pattern 1: Quantum Fourier Transform (2-qubit)")

qft_circuit = Circuit(2)
# QFT on 2 qubits
qft_circuit.h(0) # Hadamard on qubit 0
qft_circuit.rz(0, np.pi/2) # Controlled rotation
qft_circuit.cx(1, 0) # CNOT
qft_circuit.rz(0, -np.pi/2) # Undo rotation
qft_circuit.cx(1, 0) # CNOT
qft_circuit.h(1) # Hadamard on qubit 1

drawer = ASCIIDrawer(qft_circuit)
print("QFT Circuit:")
print(drawer.draw())

qft_state = executor.run(qft_circuit)
print(f"QFT result: {qft_state.data}")

In [ ]:
# Pattern 2: Controlled Operations
print("\nINTERACTIVE: Pattern 2: Controlled Operations")

# Toffoli gate (CCX) - double controlled NOT
toffoli_circuit = Circuit(3)
toffoli_circuit.h(0) # Control 1 in superposition
toffoli_circuit.h(1) # Control 2 in superposition
toffoli_circuit.ccx(0, 1, 2) # Toffoli: flips target if both controls are |1

drawer = ASCIIDrawer(toffoli_circuit)
print("Toffoli (CCX) Circuit:")
print(drawer.draw())

toffoli_state = executor.run(toffoli_circuit)
measurements = toffoli_state.measure_all(shots=1000)
print(f"Measurements: {measurements}")
print("Notice: Target qubit (2) flips only when both controls are |1")

## TARGET: Practice Exercises

### Exercise 1: Create a 4-qubit entangled state
Create a circuit where all 4 qubits are entangled (like GHZ but with 4 qubits)

In [ ]:
# Exercise 1: 4-qubit entangled state
# Your solution here

exercise1_circuit = Circuit(4)
exercise1_circuit.h(0) # Superposition on first qubit
exercise1_circuit.cx(0, 1) # Entangle 0-1
exercise1_circuit.cx(1, 2) # Entangle 1-2 
exercise1_circuit.cx(2, 3) # Entangle 2-3

print("TARGET: Exercise 1: 4-qubit entangled state")
drawer = ASCIIDrawer(exercise1_circuit)
print(drawer.draw())

state4 = executor.run(exercise1_circuit)
measurements4 = exercise1_state.measure_all(shots=1000)
print(f"Measurements: {measurements4}")
print("Expected: Only |0000 and |1111 should appear!")

### Exercise 2: Quantum SWAP gate
Create a circuit that swaps the states of two qubits

In [ ]:
# Exercise 2: SWAP gate using CNOT gates
# SWAP can be implemented with 3 CNOT gates

exercise2_circuit = Circuit(2)
# Prepare different states on the qubits
exercise2_circuit.x(0) # Qubit 0 in |1
exercise2_circuit.h(1) # Qubit 1 in |+

print("TARGET: Exercise 2: Before SWAP")
before_swap = executor.run(exercise2_circuit)
print(f"State before: {before_swap.data}")

# Implement SWAP using 3 CNOTs
exercise2_circuit.cx(0, 1) # CNOT 1
exercise2_circuit.cx(1, 0) # CNOT 2 
exercise2_circuit.cx(0, 1) # CNOT 3

print("\nSWAP Circuit:")
drawer = ASCIIDrawer(exercise2_circuit)
print(drawer.draw())

after_swap = executor.run(exercise2_circuit)
print(f"State after: {after_swap.data}")
print("The qubit states should be swapped!")

## SUCCESS: Congratulations!

You've mastered quantum circuit construction! You now know:

SUCCESS: **Multi-qubit systems** and exponential state spaces 
SUCCESS: **Two-qubit gates** (CNOT, CZ, SWAP) 
SUCCESS: **Entanglement** creation and analysis 
SUCCESS: **Bell states** and GHZ states 
SUCCESS: **Quantum teleportation** protocol 
SUCCESS: **Circuit analysis** and optimization 
SUCCESS: **Design patterns** for quantum algorithms 

## COMPASS: Next Steps

Ready for quantum algorithms? Continue with:

1. **[03_quantum_algorithms.ipynb](03_quantum_algorithms.ipynb)** - Famous quantum algorithms
2. **[04_advanced_quantum.ipynb](04_advanced_quantum.ipynb)** - Advanced topics and research

**You're becoming a quantum circuit architect!** 